## 비상벨

In [1]:
from geoband.API import *
import pandas as pd
import folium
import json
import geopandas as gpd

import re
import numpy as np

import datetime
from pyproj import Geod
from shapely import wkt
from shapely.geometry import *
from fiona.crs import *

/opt/app-root/lib/python3.6/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
police_border = gpd.read_file('./data/1.5개시도_경찰서_관할경계.geojson')
police_border.head(3)

,NAME,PNAME,geometry
0,세종경찰서,충남청,"MULTIPOLYGON (((127.17202 36.73106, 127.17202 ..."
1,진주경찰서,경남청,"MULTIPOLYGON (((128.26697 35.12927, 128.26697 ..."
2,창원서부경찰서,경남청,"MULTIPOLYGON (((128.63363 35.22152, 128.63357 ..."


In [3]:
emergency_bell = gpd.read_file('./data/22.5개시도_비상벨현황.geojson')
emergency_bell.head(3)

,bell_us,bell_set_fac,bell_loc,bell_set_yr,geometry
0,기타,화장실,오가낭뜰 근린공원(간이),2017,MULTIPOINT (127.24880 36.51230)
1,기타,화장실,어진동음악분수,2017,MULTIPOINT (127.25660 36.50190)
2,기타,화장실,제천(간이),2017,MULTIPOINT (127.25660 36.50190)


In [4]:
from tqdm import tqdm
tqdm.pandas()

In [5]:
#결측치가 없음
emergency_bell.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 18334 entries, 0 to 18333
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   bell_us       18334 non-null  object  
 1   bell_set_fac  18334 non-null  object  
 2   bell_loc      18333 non-null  object  
 3   bell_set_yr   18334 non-null  int64   
 4   geometry      18334 non-null  geometry
dtypes: geometry(1), int64(1), object(3)
memory usage: 716.3+ KB


In [6]:
# 경찰서 데이터와 결합
def detect_police_name(geometry) :
    for i in range(len(police_border['NAME'].unique())) :    # 보안등 위치의 geometry 와 관할서의 geometry를 비교하여
        if geometry.within(police_border.iloc[i].geometry) :             #  매칭된 관할서명을 return하는 함수
            return police_border.iloc[i].NAME      

In [7]:
emergency_bell['jur_stn'] = '' # 함수 적용
emergency_bell['jur_stn'] = emergency_bell.progress_apply(lambda x : detect_police_name(x['geometry']) , axis=1 )

100%|██████████| 18334/18334 [07:13<00:00, 42.28it/s]


In [8]:
emergency_bell = emergency_bell.rename({'bell_set_yr':'year'},axis=1)

In [9]:
emergency_bell

,bell_us,bell_set_fac,bell_loc,year,geometry,jur_stn
0,기타,화장실,오가낭뜰 근린공원(간이),2017,MULTIPOINT (127.24880 36.51230),세종경찰서
1,기타,화장실,어진동음악분수,2017,MULTIPOINT (127.25660 36.50190),세종경찰서
2,기타,화장실,제천(간이),2017,MULTIPOINT (127.25660 36.50190),세종경찰서
3,기타,화장실,도시공원1(남리도시 공원화장실1),2017,MULTIPOINT (127.30260 36.59780),세종경찰서
4,기타,화장실,부강면 전통시장 화장실,2017,MULTIPOINT (127.36740 36.52950),세종경찰서
...,...,...,...,...,...,...
18329,방범용,공원,북한산도시자연공원(홍은동 8-637일대),2014,MULTIPOINT (126.95000 37.60580),서울서대문경찰서
18330,방범용,화장실,참샘골공원 공중화장실,2017,MULTIPOINT (126.97470 37.61520),서울종로경찰서
18331,방범용,화장실,구기동공중화장실,2017,MULTIPOINT (126.95640 37.60960),서울종로경찰서
18332,방범용,기타,CCTV,2018,MULTIPOINT (126.94980 37.60520),서울서대문경찰서


In [10]:
# 연도 나눠주기 ( 상하반기가 나눠지지 않은 데이터이기 때문에 2016(이전), 2017(하반기) 이런형식으로 만든다)
condition = [(emergency_bell['year'] < 2017),
            (emergency_bell['year'] == 2017),
            (emergency_bell['year'] == 2018),
            (emergency_bell['year'] == 2019),
            (emergency_bell['year'] == 2020) ]
result = ['2016(이전)', '2017(하반기)', '2018(하반기)','2019(하반기)','2020(하반기)']
emergency_bell['year1'] = np.select(condition, result, default = '')

In [11]:
emergency_bell

,bell_us,bell_set_fac,bell_loc,year,geometry,jur_stn,year1
0,기타,화장실,오가낭뜰 근린공원(간이),2017,MULTIPOINT (127.24880 36.51230),세종경찰서,2017(하반기)
1,기타,화장실,어진동음악분수,2017,MULTIPOINT (127.25660 36.50190),세종경찰서,2017(하반기)
2,기타,화장실,제천(간이),2017,MULTIPOINT (127.25660 36.50190),세종경찰서,2017(하반기)
3,기타,화장실,도시공원1(남리도시 공원화장실1),2017,MULTIPOINT (127.30260 36.59780),세종경찰서,2017(하반기)
4,기타,화장실,부강면 전통시장 화장실,2017,MULTIPOINT (127.36740 36.52950),세종경찰서,2017(하반기)
...,...,...,...,...,...,...,...
18329,방범용,공원,북한산도시자연공원(홍은동 8-637일대),2014,MULTIPOINT (126.95000 37.60580),서울서대문경찰서,2016(이전)
18330,방범용,화장실,참샘골공원 공중화장실,2017,MULTIPOINT (126.97470 37.61520),서울종로경찰서,2017(하반기)
18331,방범용,화장실,구기동공중화장실,2017,MULTIPOINT (126.95640 37.60960),서울종로경찰서,2017(하반기)
18332,방범용,기타,CCTV,2018,MULTIPOINT (126.94980 37.60520),서울서대문경찰서,2018(하반기)


In [12]:
emergency_bell = emergency_bell.drop('year',axis=1)

In [13]:
emergency_bell = emergency_bell.rename({'year1':'year'}, axis=1)
emergency_bell

,bell_us,bell_set_fac,bell_loc,geometry,jur_stn,year
0,기타,화장실,오가낭뜰 근린공원(간이),MULTIPOINT (127.24880 36.51230),세종경찰서,2017(하반기)
1,기타,화장실,어진동음악분수,MULTIPOINT (127.25660 36.50190),세종경찰서,2017(하반기)
2,기타,화장실,제천(간이),MULTIPOINT (127.25660 36.50190),세종경찰서,2017(하반기)
3,기타,화장실,도시공원1(남리도시 공원화장실1),MULTIPOINT (127.30260 36.59780),세종경찰서,2017(하반기)
4,기타,화장실,부강면 전통시장 화장실,MULTIPOINT (127.36740 36.52950),세종경찰서,2017(하반기)
...,...,...,...,...,...,...
18329,방범용,공원,북한산도시자연공원(홍은동 8-637일대),MULTIPOINT (126.95000 37.60580),서울서대문경찰서,2016(이전)
18330,방범용,화장실,참샘골공원 공중화장실,MULTIPOINT (126.97470 37.61520),서울종로경찰서,2017(하반기)
18331,방범용,화장실,구기동공중화장실,MULTIPOINT (126.95640 37.60960),서울종로경찰서,2017(하반기)
18332,방범용,기타,CCTV,MULTIPOINT (126.94980 37.60520),서울서대문경찰서,2018(하반기)
